KoBERT를 활용한 IPC 코드 예측기
===================

Setup
--------

### Import packages

In [2]:
import warnings  # ignore warnings

warnings.filterwarnings("ignore")

from torch.utils.data import Dataset
import torch
import torch.nn as nn
import torch.nn.functional as F
import gluonnlp as nlp
import numpy as np
import datetime
import ast
import matplotlib.pyplot as plt
import pandas as pd
import os
import random

from tqdm.auto import tqdm # notebook 사용시 적용


from KoBERT.kobert.pytorch_kobert import get_pytorch_kobert_model
from KoBERT.kobert.utils import get_tokenizer

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from sklearn.metrics import accuracy_score

### Define class

In [3]:
# Validation loss에 따른 조기종료 설정
class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""

    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt', trace_func=print):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement.
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
            path (str): Path for the checkpoint to be saved to.
                            Default: 'checkpoint.pt'
            trace_func (function): trace print function.
                            Default: print
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path
        self.trace_func = trace_func

    def __call__(self, val_loss, model):
        
        # val_loss가 낮을 수록 좋은 모델 = 음수를 곱해 score로 사용 (score는 높을 수록 좋음)
        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            self.trace_func(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        # validation loss가 감소하면 체크포인트 저장
        if self.verbose:
            self.trace_func(
                f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

In [4]:
# BERT가 쓸 수 있는 데이터셋으로 바꿔주는 함수
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [i[label_idx] for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i],))

    def __len__(self):
        return (len(self.labels))

In [5]:
# KoBERT 예제 코드
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size=768,
                 num_classes=85,  # 데이터의 클래스 수로 조정
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size, num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids=token_ids, token_type_ids=segment_ids.long(),
                              attention_mask=attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        else:
            out = pooler
        return self.classifier(out)

In [6]:
class BalancedFocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2, reduction='mean'):
        super(BalancedFocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, logits, targets):
        # Calculate BCE loss
        bce_loss = F.binary_cross_entropy_with_logits(logits, targets, reduction='none')

        # Calculate focal weights
        focal_weights = torch.where(targets == 1, self.alpha * (1 - torch.sigmoid(logits))**self.gamma, (1 - self.alpha) * torch.sigmoid(logits)**self.gamma)

        # Apply focal weights to BCE loss
        balanced_focal_loss = focal_weights * bce_loss

        # Apply reduction
        if self.reduction == 'mean':
            return balanced_focal_loss.mean()
        elif self.reduction == 'sum':
            return balanced_focal_loss.sum()
        else:
            return balanced_focal_loss

### Define function

In [7]:
# 길이, 단어 수 등 계산
def calculate_length_stats(column):
    max_len = column.str.len().max()
    min_len = column.str.len().min()
    avg_len = column.str.len().mean()
    max_words = column.str.split().apply(len).max()
    min_words = column.str.split().apply(len).min()
    avg_words = column.str.split().apply(len).mean()
    return max_len, min_len, avg_len, max_words, min_words, avg_words


# logit을 0과 1 사이 확률 값으로 변환해주는 시그모이드
def sigmoid(arr):
    result = 1 / (1 + np.exp(-arr))
    return result

In [9]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)



seed_everything(42)  # Seed 고정

Preprocess
-----

### Import and preprocess data

In [10]:
print(f'Starting at {datetime.datetime.now()}')  # 시작 시간
print('================')
print('================\n')
# check GPU
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("GPU를 사용합니다.")
else:
    device = torch.device("cpu")
    print("CPU를 사용합니다.")

# 데이터 불러오기
data_a = pd.read_csv("pp_b.csv")
# 불러온 데이터의 ipc코드 column의 값들을 list로 변경경
data_a['ipc코드'] = data_a['ipc코드'].apply(ast.literal_eval)

# ipc코드 목록 불러오기
ipc_file_name = "ipc_b.txt"
ipc_a_dict = []
with open(ipc_file_name, "r") as file:
    for line in file:
        # Remove any leading/trailing whitespace and append the item to the list
        item = line.strip()
        ipc_a_dict.append(item)

print(ipc_a_dict)  # 실제 ipc코드 확인
print('\n================\n')

print(data_a.info())  # 데이터 기본적인 정보 확인
print('\n================\n')
print(data_a.head())
print('\n================\n')
print(f'data example : {data_a.iloc[3, 0]}')  # 데이터 예시 확인
print('\n================\n')
print(type(data_a.iloc[3, 1]))  # 데이터 ipc코드 column이 list로 제대로 변환되었는지 확인
print('\n================\n')

Starting at 2023-12-01 05:07:17.333304

GPU를 사용합니다.
['B01D', 'B01F', 'B01J', 'B02C', 'B03C', 'B05B', 'B05C', 'B05D', 'B07B', 'B08B', 'B09B', 'B21B', 'B21C', 'B21D', 'B21J', 'B22D', 'B22F', 'B23B', 'B23C', 'B23D', 'B23K', 'B23P', 'B23Q', 'B24B', 'B24D', 'B25B', 'B25J', 'B26B', 'B26D', 'B28B', 'B29B', 'B29C', 'B29D', 'B29K', 'B29L', 'B30B', 'B31B', 'B32B', 'B33Y', 'B41F', 'B41J', 'B41M', 'B42D', 'B42F', 'B43K', 'B43L', 'B44C', 'B60B', 'B60C', 'B60G', 'B60H', 'B60J', 'B60K', 'B60L', 'B60N', 'B60P', 'B60Q', 'B60R', 'B60S', 'B60T', 'B60W', 'B61L', 'B62B', 'B62D', 'B62J', 'B62K', 'B62M', 'B63B', 'B63C', 'B63H', 'B63J', 'B64C', 'B64D', 'B65B', 'B65D', 'B65F', 'B65G', 'B65H', 'B66B', 'B66C', 'B66D', 'B66F', 'B67D', 'B82B', 'B82Y']


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172412 entries, 0 to 172411
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   초록      172412 non-null  object
 1   ipc코드   172412 non-null  object
dtypes:

### Transform data for model training

In [11]:
# data를 list로 변경, list의 element 하나가 ['초록', [ipc 코드 one-hot encoding된 list]]
data_a_list = []

for ques, label in zip(data_a['초록'], data_a['ipc코드']):
    tmp_data = []  # 위에 언급한 'list의 element 하나'
    tmp_data.append(ques)
    tmp_data.append(np.array(label))
    data_a_list.append(tmp_data)

# train & test 데이터로 나누기
from sklearn.model_selection import train_test_split

# train, test(검증용 데이터), test_final (최종 test용)
a_traval, a_test_final = train_test_split(data_a_list, test_size=0.3, random_state=9871)
a_train, a_test = train_test_split(a_traval, test_size=0.3, random_state=23)

print(f'length of train / val / test set : {len(a_train)} / {len(a_test)} / {len(a_test_final)}')
print('\n================\n')

length of train / val / test set : 84481 / 36207 / 51724



Define model
-----

### Define hyperparameters and prepare data

In [12]:
# hyperparameter
'''
max_len : BERT 모델이 한 번에 받는 최대 word token의 수 (최대 512)
batch_size : 한 번에 처리할 batch의 size
warmup_ratio : 학습률이 유동적으로 변하게 해주느 Learning rate scheduler의 hyperparameter https://ai4nlp.tistory.com/8
num_epochs : 총 학습할 epoch
max_grad_norm : 오차 역전파 과정에서 일어날 수 있는 학습 무효화 방지 https://eehoeskrap.tistory.com/582
log_interval : 몇 batch마다 accuracy 보여줄지(학습과는 상관 X, 확인용)
learning_rate : 학습률, 현재 optimizer는 Adam으로 5e-5, 2e-5, 3e-5, 1e-4 정도가 해볼만 함 https://arxiv.org/pdf/1810.04805.pdf (BERT 공식 문서 appendix에 적혀있음)
threshold : logit으로 반환된 예측 결과에 sigmoid를 적용해 확률 값으로 나타내는데, 확률 값이 threshold 이상이어야 해당 label인 것으로 판단
patience : patience만큼의 epoch 동안 validation loss가 좋아지지 않으면 학습 종료
'''
max_len = 512
batch_size = 8
warmup_ratio = 0.1
num_epochs = 15
max_grad_norm = 1 
log_interval = 10000 
learning_rate = 5e-5 
threshold = 0.5  
patience = 5

tokenizer = get_tokenizer()
# BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model(cachedir=".cache")
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

# BERTDataset : 각 데이터가 BERT 모델의 입력으로 들어갈 수 있도록 tokenization, int encoding, padding하는 함수
data_train = BERTDataset(a_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(a_test, 0, 1, tok, max_len, True, False)
data_test_final = BERTDataset(a_test_final, 0, 1, tok, max_len, True, False)

# torch 형식의 dataset을 만들어 입력 데이터셋의 전처리 마무리
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=0)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=0)
final_dataloader = torch.utils.data.DataLoader(data_test_final, batch_size=batch_size, num_workers=0)

using cached model. C:\Users\user\Desktop\capstone\.cache\kobert_news_wiki_ko_cased-1087f8699e.spiece
using cached model. C:\Users\user\Desktop\capstone\.cache\kobert_v1.zip
using cached model. C:\Users\user\Desktop\capstone\.cache\kobert_news_wiki_ko_cased-1087f8699e.spiece


### Load model, define loss function

In [13]:
# BERT 모델 불러오기
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

# optimizer와 schedule 설정
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
     'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
# loss_fn = nn.CrossEntropyLoss() # 다중분류를 위한 loss function
loss_fn = BalancedFocalLoss(alpha=0.05, gamma=2, reduction='mean')

t_total = len(train_dataloader) * num_epochs  # 전체 시행하게 될 학습 횟수
warmup_step = int(t_total * warmup_ratio)  # 전체 학습의 warmup_ratio 비율만큼 warmup 단계로 설정
print(len(train_dataloader))

# warmup 단계에 필요한 scheduler
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

10561


Model training
--------

### Train and validate

In [14]:

train_history = []
test_history = []
loss_history = []

# epoch 당 평균 loss 기록 (training)
avg_train_losses = []
# epoch 당 평균 loss 기록 (validation)
avg_valid_losses = []

es_count = 0

early_stopping = EarlyStopping(patience=patience, verbose=True, path='checkpoint.pt')
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    count_acc_tr = 0
    count_total_tr = 0
    count_acc_test = 0
    count_total_test = 0
    model.train()
    train_epoch_pred = []
    train_loss_record = []
    # epoch 내에서 training loss 기록
    train_losses = []
    # epoch 내에서 validation loss 기록
    valid_losses = []


    # Train
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(train_dataloader),
                                                                        total=len(train_dataloader)):
        optimizer.zero_grad() # optimizer의 gradient 0으로 초기화 

        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length = valid_length

        label = label.float().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)

        train_loss_record.append(loss)

        train_pred = out.detach().cpu().numpy()
        train_pred = sigmoid(train_pred)  # logit에 sigmoid 적용해서 0과 1 사이(확률)로 변환
        train_pred = (train_pred >= threshold).astype(float)  # threshold 기준으로 label 판단
        train_real = label.detach().cpu().numpy()

        if batch_id % log_interval == 0:
            train_batch_result = accuracy_score(np.array(train_real), np.array(train_pred))
            print(
                f"epoch {e}, batch number {batch_id}, train label-wise accuracy is : {train_batch_result}")
        train_epoch_pred.append(train_pred)

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule

        train_losses.append(loss.item())

    train_epoch_pred = np.concatenate(train_epoch_pred)
    train_epoch_target = train_dataloader.dataset.labels
    train_epoch_result = accuracy_score(y_true=train_epoch_target, y_pred=train_epoch_pred)


    print(f"=====Training Report: Epoch {e} mean loss is {sum(train_loss_record) / len(train_loss_record)}=====")
    print(f'Accuracy is {train_epoch_result}')
    model.eval() # 학습 모드가 아닌 예측 모드로 변경

    # Validation
    with torch.no_grad(): # gradient 계산 X로 추론 속도 빨라짐
        test_epoch_pred = []
        test_loss_record = []
        test_acc = 0.0
        for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(test_dataloader),
                                                                            total=len(test_dataloader)):
            token_ids = token_ids.long().to(device)
            segment_ids = segment_ids.long().to(device)
            valid_length = valid_length

            label = label.float().to(device)
            out = model(token_ids, valid_length, segment_ids)
            loss = loss_fn(out, label)

            test_loss_record.append(loss)

            test_pred = out.detach().cpu().numpy()
            test_pred = sigmoid(test_pred)
            test_pred = (test_pred >= threshold).astype(float)
            test_real = label.detach().cpu().numpy()

            valid_losses.append(loss.item())


            test_batch_result = accuracy_score(np.array(test_real), np.array(test_pred))

            test_epoch_pred.append(test_pred)

        test_epoch_pred = np.concatenate(test_epoch_pred)
        test_epoch_target = test_dataloader.dataset.labels
        test_epoch_result = accuracy_score(y_true=test_epoch_target, y_pred=test_epoch_pred)


        print(f"=====Validation Report: mean loss is {sum(test_loss_record) / len(test_loss_record)}=====")
        print(f'Accuracy is {test_epoch_result}')


        valid_loss = np.average(valid_losses)
        train_loss = np.average(train_losses)
        avg_valid_losses.append(valid_loss)
        avg_train_losses.append(train_loss)
        
        train_history.append(train_epoch_result)
        test_history.append(test_epoch_result)
        
        if e > 0:
            if test_history[e] < test_history[e-1]:
                es_count += 1
            else:
                es_count = 0
                torch.save(model.state_dict(), 'checkpoint.pt')
        if es_count == 3:
            break

        # 조기 종료 체크
        """
        early_stopping(valid_loss, model)
        if early_stopping.early_stop:
            print('Early stopping!')
            break
            """


# 시각화
plt.plot(avg_train_losses, label='Train Loss')
plt.plot(avg_valid_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.savefig('loss_a.png')
plt.close()

print(f'train acc : {train_history}')
print(f'test acc : {test_history}')

  0%|          | 0/10561 [00:00<?, ?it/s]

epoch 0, batch number 0, train label-wise accuracy is : 0.0
epoch 0, batch number 10000, train label-wise accuracy is : 0.0
=====Training Report: Epoch 0 mean loss is 0.013499485328793526=====
Accuracy is 0.00010653282986707071


  0%|          | 0/4526 [00:00<?, ?it/s]

=====Validation Report: mean loss is 0.0014008455909788609=====
Accuracy is 0.0


  0%|          | 0/10561 [00:00<?, ?it/s]

...

=====Validation Report: mean loss is 0.001681844936683774=====
Accuracy is 0.5958792498688099


  0%|          | 0/10561 [00:00<?, ?it/s]

epoch 14, batch number 0, train label-wise accuracy is : 1.0
epoch 14, batch number 10000, train label-wise accuracy is : 0.75
=====Training Report: Epoch 14 mean loss is 0.0001902539370348677=====
Accuracy is 0.8146447130123933


  0%|          | 0/4526 [00:00<?, ?it/s]

=====Validation Report: mean loss is 0.0016979699721559882=====
Accuracy is 0.5994144778633966
train acc : [0.00010653282986707071, 0.032587208958227296, 0.1265846758442727, 0.2016784839194612, 0.2684745682461145, 0.3412956759508055, 0.4172772576082196, 0.4969283034054995, 0.5698559439400576, 0.6378238893952486, 0.7013056190149264, 0.7496123388690948, 0.7861649364945964, 0.8067967945455191, 0.8146447130123933]
test acc : [0.0, 0.06653409561686967, 0.18170519512801392, 0.22382412240726932, 0.3419780705388461, 0.3602618278233491, 0.40351313281962053, 0.4124892976496258, 0.49636810561493633, 0.5243737398845527, 0.5625155356698981, 0.5751097854006132, 0.5912116441572072, 0.5958792498688099, 0.5994144778633966]


### Test model on test dataset

In [15]:
model.load_state_dict(torch.load('checkpoint.pt'))
torch.save(model.state_dict(), 'bert_b.pth')
model.eval()

with torch.no_grad():
    final_epoch_pred = []
    final_loss_record = []
    # test done per epoch
    final_test_acc = 0.0
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(final_dataloader),
                                                                        total=len(final_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length = valid_length

        label = label.float().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)

        final_loss_record.append(loss)

        f_pred = out.detach().cpu().numpy()
        f_pred = sigmoid(f_pred)
        f_pred = (f_pred >= threshold).astype(float)
        f_real = label.detach().cpu().numpy()

        final_batch_result = accuracy_score(np.array(f_real), np.array(f_pred))

        final_epoch_pred.append(f_pred)

    final_epoch_pred = np.concatenate(final_epoch_pred)
    final_epoch_target = final_dataloader.dataset.labels
    final_epoch_result = accuracy_score(y_true=final_epoch_target, y_pred=final_epoch_pred)


    print(f"=====Final Testing Report: mean loss is {sum(final_loss_record) / len(final_loss_record)}=====")
    print(f'Accuracy is {final_epoch_result}')

  0%|          | 0/6466 [00:00<?, ?it/s]

=====Final Testing Report: mean loss is 0.0016765096224844456=====
Accuracy is 0.5968022581393551
